In [4]:
import os
import json
import pandas as pd

from glob import glob

In [5]:
base_dir = os.path.join(os.path.expanduser("~"), "data", "ofij")
if not os.path.exists(base_dir):
    os.makedirs(base_dir)


In [9]:

news_files = glob(os.path.join(base_dir, 'news*.feather'))
newsembed_file = os.path.join(base_dir, 'batch_embedding_output.jsonl')
stmeta_file = os.path.join(base_dir,'stock_meta.feather')
stkmeta_file = os.path.join(base_dir, 'stock_meta_response.jsonl')

In [35]:
# load jsonl file 
with open(newsembed_file, 'r') as f:
    newsembed = [json.loads(line) for line in f]

# extract custom_id and embedding from newsembed
dfemb = []
for newsrec in newsembed:
    custom_id = newsrec['custom_id']
    embedding = newsrec['response']['body']['data'][0]['embedding']
    dfemb.append((custom_id, embedding))
dfemb = pd.DataFrame(dfemb, columns=['custom_id', 'embedding'])

# remove newsembed object from memory
del newsembed


In [20]:
dfmeta = pd.read_feather(stmeta_file).reset_index()
# convert index column to string
dfmeta['index'] = dfmeta['index'].astype(str)

In [14]:
# load stkmeta_file jsonl file
with open(stkmeta_file, 'r') as f:
    stkmeta = [json.loads(line) for line in f]
dfstsent = [(res['custom_id'], res['response']['body']['choices'][0]['message']['content'] ) for res in stkmeta]
dfstsent = pd.DataFrame(dfstsent, columns=['index', 'stksent'])


In [22]:
dfmeta = dfmeta.merge(dfstsent, how='left', on='index')

In [36]:
dfnews = pd.read_feather(news_files[0])

In [37]:
dfnews = dfnews.drop_duplicates().reset_index(drop=False)

In [38]:
# convert index column to string
dfnews['index'] = dfnews['index'].astype(str)
dfnews.head()

,index,cntt_usiq_srno,news_ofer_entp_code,data_dt,data_tm,hts_pbnt_titl_cntt,news_lrdv_code,dorg,iscd1,kor_isnm1
0,0,2024032100100098155,U,20240321,001000,[기자의 눈] 벚꽃없는 벚꽃축제,39,서울경제,,
1,1,2024032100090896253,2,20240321,000908,"외신들,""엔비디아 젠슨황이 AI계 스티브 잡스"" 평가",04,한국경제신문,,
2,2,2024032100053623852,2,20240321,000536,베트남 GDP 3% 횡령한 부동산 재벌…사형 구형,09,한국경제신문,,
3,3,2024032100050072449,U,20240321,000500,[사설] “정규직 과보호에 중장년 고용 불안”···노동 유연화 서둘러야,39,서울경제,,
4,4,2024032100050059151,U,20240321,000500,"[사설] 의대별 정원 확정, 특위에서 필수?지역 의료 정상화에 머리 맞대라",39,서울경제,,


In [39]:
# merge dfnews and dfemb on index column and custom_id
dfemb = dfnews.merge(dfemb, left_on='index', right_on='custom_id', how='inner')
dfemb.head(1)

,index,cntt_usiq_srno,news_ofer_entp_code,data_dt,data_tm,hts_pbnt_titl_cntt,news_lrdv_code,dorg,iscd1,kor_isnm1,custom_id,embedding
0,0,2024032100100098155,U,20240321,001000,[기자의 눈] 벚꽃없는 벚꽃축제,39,서울경제,,,0,"[0.02361799, 0.007665091, -0.009329446, -0.016..."


In [40]:
dfemb.drop(columns=['custom_id', 'iscd1','kor_isnm1'], inplace=True, errors='ignore')

In [41]:
dfemb.head()

,index,cntt_usiq_srno,news_ofer_entp_code,data_dt,data_tm,hts_pbnt_titl_cntt,news_lrdv_code,dorg,embedding
0,0,2024032100100098155,U,20240321,001000,[기자의 눈] 벚꽃없는 벚꽃축제,39,서울경제,"[0.02361799, 0.007665091, -0.009329446, -0.016..."
1,1,2024032100090896253,2,20240321,000908,"외신들,""엔비디아 젠슨황이 AI계 스티브 잡스"" 평가",04,한국경제신문,"[0.01607047, -0.0054703546, 0.0006299249, 0.01..."
2,2,2024032100053623852,2,20240321,000536,베트남 GDP 3% 횡령한 부동산 재벌…사형 구형,09,한국경제신문,"[0.029173603, 0.020256389, 0.060813203, 0.0384..."
3,3,2024032100050072449,U,20240321,000500,[사설] “정규직 과보호에 중장년 고용 불안”···노동 유연화 서둘러야,39,서울경제,"[0.033023026, 0.077666745, 0.0033049677, 0.037..."
4,4,2024032100050059151,U,20240321,000500,"[사설] 의대별 정원 확정, 특위에서 필수?지역 의료 정상화에 머리 맞대라",39,서울경제,"[-0.022171568, 0.0067736073, 0.014170361, 0.04..."


In [42]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
texts = dfemb.hts_pbnt_titl_cntt.to_list()
embeddings = np.array(dfemb.embedding.to_list())

In [44]:
similarity_mtx = cosine_similarity(embeddings)

In [ ]:
similarity_mtx

(10401, 10401)

In [62]:
similarity_mtx[0].argsort()[::-1]

array([   0, 3416, 7422, ..., 1878, 1920, 2245], dtype=int64)

In [63]:
texts[3416]

"'3월 벚꽃축제' 안양천서 열린다"

In [64]:
texts[7422]

'벚꽃·유채꽃·철쭉 흐드러진 경남, 봄꽃 축제 보러 오세요'

### Mapping stock meta to news

In [51]:
import httpx
import openai

from openai import OpenAI
from dotenv import load_dotenv

In [52]:
load_dotenv()
httpx_client= httpx.Client(verify=False) #, timeout=60)
openai.api_key  = os.environ['OPENAI_API_KEY']
client = OpenAI(http_client=httpx_client)

In [90]:
stksentence = dfstsent.iloc[40,1]
stksentence

'SK하이닉스는 반도체 제조업에 속하는 전기,전자 업종의 시가총액 규모대 기업입니다.'

In [91]:
response_1 = client.embeddings.create(
    input=stksentence,
    model="text-embedding-3-small",
    encoding_format="float"
)

In [92]:
stkemb = np.array(response_1.data[0].embedding)

In [93]:
embsearch_res = cosine_similarity(embeddings, stkemb.reshape(1, -1)).flatten()

In [94]:
texts[embsearch_res.argmax()]

'SK하이닉스, 내년 3월 세계 최대 반도체 공장 짓는다'

In [95]:
for embi in embsearch_res.argsort()[::-1][:10]:
    print(f"Similarity: {embsearch_res[embi]:.4f} - {texts[embi]}")

Similarity: 0.6279 - SK하이닉스, 내년 3월 세계 최대 반도체 공장 짓는다
Similarity: 0.5738 - SK하이닉스 최첨단 반도체 생산기지 내년 3월 착공…2027년 완공
Similarity: 0.5197 - SK하이닉스(000660)  +5.05%, 삼성전자 +2.60%, DB하이텍 +1.01%
Similarity: 0.4939 - 미국發 반도체 호재 만발…삼성전자·SK하이닉스 '강세'
Similarity: 0.4874 - SK하이닉스, 본격 AI 시대 발맞춰 … 고성능·고용량 메모리 양산 박차
Similarity: 0.4781 - 마이크론 훈풍에 SK하이닉스 8%삼성전자 3%…반도체 동반강세(종합)
Similarity: 0.4720 - SK하이닉스, 용인 반도체 클러스터 내년 3월 착공
Similarity: 0.4398 - LG화학, 모로코 산업단지서 '해수 담수화' 역삼투막 공급
Similarity: 0.4396 - ‘반도체 풍향계’ 마이크론 매출 급등에 주가 15% 껑충…하이닉스·삼성전자도 동반상승
Similarity: 0.4345 - LG화학, 세계 최대 비료단지에 해수담수화 역삼투막 공급
